# Bike Data Processing - Complete Pipeline
This notebook:
1. Loads the Excel file with bike crash data
2. Filters to rows with Bike Type values
3. Extracts ZIP files containing crash report PDFs
4. Matches HSMV Report Numbers with PDF files
5. Extracts narrative text from matching PDFs
6. Generates statistics and saves results

In [ ]:
# Import required libraries
!pip install PyPDF2
import pandas as pd
import numpy as np
import os
import re
import zipfile
from pathlib import Path
import PyPDF2

In [ ]:
# Configuration - adjust paths if needed
excel_file = "nolabeluse.xlsx"
reports_folder = "/content/drive/MyDrive/reports"
output_file = "bike_data_with_narratives.xlsx"

print(f"Excel file: {excel_file}")
print(f"Reports folder: {reports_folder}")
print(f"Output file: {output_file}")

Excel file: nolabeluse.xlsx
Reports folder: reports
Output file: bike_data_with_narratives.xlsx


## Step 1: Load and Filter Excel Data

In [38]:
# Read the Excel file
df = pd.read_excel(excel_file)

print(f"Original data shape: {df.shape}")
print(f"\nColumns in original file:")
for col in df.columns:
    print(f"  - {col}")

Original data shape: (9031, 45)

Columns in original file:
  - HSMV_Report_Number
  - Reporting_Agency
  - Form_Type
  - Year
  - Crash_Date
  - Crash_Time
  - City
  - County
  - Crash_Street
  - Intersecting_Street
  - Vehicles
  - Non_Motorists
  - Fatalities
  - Injuries
  - Alcohol_Related
  - Distraction_Related
  - Drug_Related
  - Weather_Condition
  - Light_Condition
  - Crash_Severity
  - Type_of_Intersection
  - Road_Sys_Identifier
  - Type_of_Shoulder
  - Road_Surf_Cond
  - Bicyclists
  - Possible_Injuries
  - Non_Incapacitating_Injuries
  - Incapacitating_Injuries
  - Fatalities_30_Days
  - Non_Traffic_Fatalities
  - S4_Mapping
  - S4_Decimal_Degree_Longitude
  - S4_Decimal_Degree_Latitude
  - S4_Albers_X
  - S4_Albers_Y
  - S4_Mapping_Date
  - Bike_Crash_Group_Number
  - Bike_Crash_Group
  - Bike_Crash_Type_Number
  - Bike_Crash_Type
  - Bike_Crash_Location
  - Bike_Bicyclist_Direction
  - Bike_Bicyclist_Position
  - Bike_Typing_Notes
  - Bike Type


In [39]:
# Create filtered dataframe with required columns
df_filtered = df[['HSMV_Report_Number', 'Bike Type']].copy()

# Add empty Narrative and Match columns
df_filtered['Narrative'] = ''
df_filtered['Match'] = 'No'

# Filter: Keep only rows where 'Bike Type' has a value
df_filtered = df_filtered[df_filtered['Bike Type'].notna()]
df_filtered = df_filtered[df_filtered['Bike Type'].astype(str).str.strip() != '']

print(f"\nFiltered data shape: {df_filtered.shape}")
print(f"Rows kept: {len(df_filtered)}")
print(f"Rows removed: {len(df) - len(df_filtered)}")

print("\nFirst few rows:")
df_filtered.head()


Filtered data shape: (111, 4)
Rows kept: 111
Rows removed: 8920

First few rows:


,HSMV_Report_Number,Bike Type,Narrative,Match
403,82089001,Motorized bicycle,,No
530,81994356,Motorized bicycle,,No
745,82877472,E-trike,,No
927,83350430,Pedicab,,No
1031,83353372,Motorized bicycle,,No


## Step 2: Extract ZIP Files

In [40]:
# Get list of ZIP files in reports folder
zip_files = [f for f in os.listdir(reports_folder) if f.endswith('.zip')]
print(f"Found {len(zip_files)} ZIP files in {reports_folder}")

# Show first few
if zip_files:
    print(f"\nFirst 3 ZIP files:")
    for zf in zip_files[:3]:
        print(f"  - {zf}")

Found 29 ZIP files in reports

First 3 ZIP files:
  - S4_Extract_xiangyan_145031_20250507_221605.zip
  - S4_Extract_xiangyan_145032_20250507_221817.zip
  - S4_Extract_xiangyan_145033_20250507_222016.zip


In [41]:
# Extract all ZIP files (this may take a few minutes)
print("Extracting ZIP files...")
extracted_count = 0

for zip_file in zip_files:
    zip_path = os.path.join(reports_folder, zip_file)
    extract_folder = os.path.join(reports_folder, zip_file.replace('.zip', ''))
    
    # Check if already extracted
    if not os.path.exists(extract_folder):
        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_folder)
            extracted_count += 1
            if extracted_count % 10 == 0:
                print(f"  Extracted {extracted_count}/{len(zip_files)} files...")
        except Exception as e:
            print(f"  Error extracting {zip_file}: {e}")
    else:
        print(f"  Already extracted: {zip_file}")

print(f"\nExtraction complete! Extracted {extracted_count} new files.")

Extracting ZIP files...
  Already extracted: S4_Extract_xiangyan_145031_20250507_221605.zip
  Already extracted: S4_Extract_xiangyan_145032_20250507_221817.zip
  Already extracted: S4_Extract_xiangyan_145033_20250507_222016.zip
  Already extracted: S4_Extract_xiangyan_145034_20250507_222144.zip
  Already extracted: S4_Extract_xiangyan_145035_20250507_222250.zip
  Already extracted: S4_Extract_xiangyan_145036_20250507_222415.zip
  Already extracted: S4_Extract_xiangyan_145037_20250507_222521.zip
  Already extracted: S4_Extract_xiangyan_145038_20250507_222618.zip
  Already extracted: S4_Extract_xiangyan_145039_20250507_224519.zip
  Already extracted: S4_Extract_xiangyan_145040_20250507_224607.zip
  Already extracted: S4_Extract_xiangyan_145041_20250507_224750.zip
  Already extracted: S4_Extract_xiangyan_145042_20250507_224908.zip
  Already extracted: S4_Extract_xiangyan_145043_20250507_225718.zip
  Already extracted: S4_Extract_xiangyan_145044_20250507_225800.zip
  Already extracted: S4_

## Step 3: Build PDF Mapping

In [42]:
# Find all extracted folders
extracted_folders = [f for f in os.listdir(reports_folder) 
                     if os.path.isdir(os.path.join(reports_folder, f))]

print(f"Found {len(extracted_folders)} extracted folders")
print(f"\nFirst few folders:")
for folder in extracted_folders[:3]:
    print(f"  - {folder}")

Found 29 extracted folders

First few folders:
  - S4_Extract_xiangyan_145031_20250507_221605
  - S4_Extract_xiangyan_145032_20250507_221817
  - S4_Extract_xiangyan_145033_20250507_222016


In [43]:
# Build mapping: report_number -> PDF path and ZIP folder location
report_pdf_map = {}
report_zip_map = {}  # Track which ZIP folder each report came from

for folder in extracted_folders:
    folder_path = os.path.join(reports_folder, folder)
    police_reports_path = os.path.join(folder_path, "Police Crash Reports")
    
    # Check if Police Crash Reports subfolder exists
    if os.path.exists(police_reports_path):
        pdf_files = [f for f in os.listdir(police_reports_path) if f.endswith('.pdf')]
        
        for pdf_file in pdf_files:
            # Extract report number from filename: CrashReport_85828586.pdf
            match = re.search(r'CrashReport_(\d+)\.pdf', pdf_file)
            if match:
                report_num = match.group(1)
                pdf_path = os.path.join(police_reports_path, pdf_file)
                report_pdf_map[report_num] = pdf_path
                report_zip_map[report_num] = folder  # Store the ZIP folder location

print(f"Built mapping for {len(report_pdf_map)} PDFs")
print(f"\nSample report numbers from PDFs:")
sample_keys = list(report_pdf_map.keys())[:5]
for key in sample_keys:
    print(f"  - {key}: {report_zip_map[key]}")

Built mapping for 2787 PDFs

Sample report numbers from PDFs:
  - 81421443: S4_Extract_xiangyan_145031_20250507_221605
  - 81428801: S4_Extract_xiangyan_145031_20250507_221605
  - 81443557: S4_Extract_xiangyan_145031_20250507_221605
  - 81465854: S4_Extract_xiangyan_145032_20250507_221817
  - 81527069: S4_Extract_xiangyan_145031_20250507_221605


In [44]:
# Check sample of report numbers from Excel to ensure format matches
print("Sample report numbers from Excel:")
sample_excel = df_filtered['HSMV_Report_Number'].head(10).astype(str).tolist()
for num in sample_excel:
    print(f"  - {num}")

Sample report numbers from Excel:
  - 82089001
  - 81994356
  - 82877472
  - 83350430
  - 83353372
  - 83353605
  - 83360086
  - 80823780
  - 83968397
  - 83671693


## Step 4: Match Report Numbers

In [45]:
# Function to check if report number has matching PDF
def check_match(report_number):
    # Convert to string and clean
    report_str = str(report_number).strip()
    
    # Remove any decimal points (in case of float conversion)
    if '.' in report_str:
        report_str = report_str.split('.')[0]
    
    return 'Yes' if report_str in report_pdf_map else 'No'

# Apply matching
df_filtered['Match'] = df_filtered['HSMV_Report_Number'].apply(check_match)

print("Match Results:")
print(df_filtered['Match'].value_counts())
print(f"\nMatch rate: {(df_filtered['Match'] == 'Yes').sum() / len(df_filtered) * 100:.2f}%")

Match Results:
Match
No     96
Yes    15
Name: count, dtype: int64

Match rate: 13.51%


In [46]:
# Show samples of matched and unmatched records
print("\nSample MATCHED records:")
matched = df_filtered[df_filtered['Match'] == 'Yes'].head(3)
print(matched[['HSMV_Report_Number', 'Bike Type', 'Match']])

print("\nSample UNMATCHED records:")
unmatched = df_filtered[df_filtered['Match'] == 'No'].head(3)
print(unmatched[['HSMV_Report_Number', 'Bike Type', 'Match']])


Sample MATCHED records:
      HSMV_Report_Number Bike Type Match
3756            85203537   Pedicab   Yes
4727            85442822   Pedicab   Yes
5318            87436908   Pedicab   Yes

Sample UNMATCHED records:
     HSMV_Report_Number          Bike Type Match
403            82089001  Motorized bicycle    No
530            81994356  Motorized bicycle    No
745            82877472            E-trike    No


## Step 5: Extract Narratives from PDFs

In [47]:
def extract_narrative_from_pdf(pdf_path):
    """
    Extract the NARRATIVE section from a Florida Traffic Crash Report PDF
    
    Args:
        pdf_path: Path to the PDF file
        
    Returns:
        Extracted narrative text, or empty string if not found
    """
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            
            # Extract text from all pages
            full_text = ""
            for page in pdf_reader.pages:
                full_text += page.extract_text()
            
            # Look for NARRATIVE section
            narrative_pattern = r'NARRATIVE\s*[-\s]*\n(.*?)(?=REPORTING OFFICER|$)'
            match = re.search(narrative_pattern, full_text, re.DOTALL | re.IGNORECASE)
            
            if match:
                narrative_text = match.group(1).strip()
                
                # Clean up the narrative text
                lines = narrative_text.split('\n')
                cleaned_lines = []
                
                for line in lines:
                    # Skip header lines and dash lines
                    if any(x in line for x in ['ID Number', 'Rank', 'Officer Agency', 'TROOPER', 'OFFICER']):
                        if not any(word in line.upper() for word in ['REPORTING', 'OFFICER AGENCY PHONE']):
                            continue
                    if line.strip().replace('-', '').strip() == '':
                        continue
                    
                    cleaned_lines.append(line.strip())
                
                # Join lines and clean up extra whitespace
                narrative = ' '.join(cleaned_lines)
                narrative = re.sub(r'\s+', ' ', narrative).strip()
                
                return narrative
            else:
                return ""
                
    except Exception as e:
        print(f"Error extracting from {pdf_path}: {str(e)}")
        return ""

print("PDF extraction function defined!")

PDF extraction function defined!


In [48]:
# Test extraction with one PDF
sample_report_num = list(report_pdf_map.keys())[0]
sample_pdf_path = report_pdf_map[sample_report_num]

print(f"Testing extraction with:")
print(f"  Report Number: {sample_report_num}")
print(f"  PDF Path: {sample_pdf_path}\n")

test_narrative = extract_narrative_from_pdf(sample_pdf_path)
print("Extracted Narrative:")
print("=" * 80)
print(test_narrative[:500] + "..." if len(test_narrative) > 500 else test_narrative)
print("=" * 80)

Testing extraction with:
  Report Number: 81421443
  PDF Path: reports\S4_Extract_xiangyan_145031_20250507_221605\Police Crash Reports\CrashReport_81421443.pdf

Extracted Narrative:



In [49]:
# Extract narratives for all matched reports
def get_narrative_for_report(row):
    """Get narrative for a report if it has a matching PDF"""
    if row['Match'] == 'No':
        return ''
    
    report_str = str(row['HSMV_Report_Number']).strip()
    if '.' in report_str:
        report_str = report_str.split('.')[0]
    
    if report_str in report_pdf_map:
        pdf_path = report_pdf_map[report_str]
        return extract_narrative_from_pdf(pdf_path)
    else:
        return ''

print("Extracting narratives from PDFs...")
print("This may take several minutes...\n")

# Apply extraction
df_filtered['Narrative'] = df_filtered.apply(get_narrative_for_report, axis=1)

# Count results
narratives_extracted = (df_filtered['Narrative'] != '').sum()
print(f"\nNarratives extracted: {narratives_extracted} out of {len(df_filtered)} records")
print(f"Extraction rate: {(narratives_extracted / len(df_filtered) * 100):.2f}%")

Extracting narratives from PDFs...
This may take several minutes...


Narratives extracted: 14 out of 111 records
Extraction rate: 12.61%


In [50]:
# Show sample of extracted narratives
print("Sample records with extracted narratives:\n")
sample_with_narrative = df_filtered[df_filtered['Narrative'] != ''].head(3)

for idx, row in sample_with_narrative.iterrows():
    print(f"Report Number: {row['HSMV_Report_Number']}")
    print(f"Bike Type: {row['Bike Type']}")
    print(f"Match: {row['Match']}")
    print(f"Narrative: {row['Narrative'][:200]}...")
    print("-" * 80)

Sample records with extracted narratives:

Report Number: 85203537
Bike Type: Pedicab
Match: Yes
Narrative: ID Number Rank Name Troop / Post Officer Agency Phone Number Date Created V01 and NM01 was traveling east in the left lane on Universal Blvd east of Convention Way. NM02 and NM03 were passengers on th...
--------------------------------------------------------------------------------
Report Number: 87436908
Bike Type: Pedicab
Match: Yes
Narrative: Vehicle 1 was stopped in traffic on Universal Blvd facing southbound at the intersection of Major Blvd in the right lane. The Cyclist (person 2) was directly to the right (West) of Vehicle 1 facing th...
--------------------------------------------------------------------------------
Report Number: 87441667
Bike Type: Motorized bicycle
Match: Yes
Narrative: Vehicle 1 was traveling East on E. Colonial Dr and approaching the intersection with Fashion Square Mall Entrance. Vehicle 1 was in the left turn lane. The cyclist was operating a ga

In [51]:
# Add ZIP folder location for tracking missing narratives
def get_zip_folder_location(row):
    """Get the ZIP folder location for a report"""
    if row['Match'] == 'No':
        return ''
    
    report_str = str(row['HSMV_Report_Number']).strip()
    if '.' in report_str:
        report_str = report_str.split('.')[0]
    
    if report_str in report_zip_map:
        return report_zip_map[report_str]
    else:
        return ''

print("Adding ZIP folder location column...")
df_filtered['ZIP_Folder_Location'] = df_filtered.apply(get_zip_folder_location, axis=1)

# Show statistics
zip_locations_filled = (df_filtered['ZIP_Folder_Location'] != '').sum()
print(f"\nZIP folder locations tracked: {zip_locations_filled} out of {len(df_filtered)} records")
print(f"\nSample records with ZIP folder location:")
print(df_filtered[df_filtered['ZIP_Folder_Location'] != ''][['HSMV_Report_Number', 'Bike Type', 'Match', 'ZIP_Folder_Location']].head())

Adding ZIP folder location column...

ZIP folder locations tracked: 15 out of 111 records

Sample records with ZIP folder location:
      HSMV_Report_Number          Bike Type Match  \
3756            85203537            Pedicab   Yes   
4727            85442822            Pedicab   Yes   
5318            87436908            Pedicab   Yes   
5480            87441667  Motorized bicycle   Yes   
5531            87443630  Motorized bicycle   Yes   

                             ZIP_Folder_Location  
3756  S4_Extract_xiangyan_145031_20250507_221605  
4727  S4_Extract_xiangyan_145032_20250507_221817  
5318  S4_Extract_xiangyan_145034_20250507_222144  
5480  S4_Extract_xiangyan_145034_20250507_222144  
5531  S4_Extract_xiangyan_145034_20250507_222144  


## Step 6: Statistics and Export

In [52]:
# Comprehensive statistics
print("=" * 80)
print("FINAL STATISTICS")
print("=" * 80)

print(f"\nTotal records: {len(df_filtered)}")
print(f"Unique HSMV Report Numbers: {df_filtered['HSMV_Report_Number'].nunique()}")
print(f"Unique Bike Types: {df_filtered['Bike Type'].nunique()}")

print("\n" + "-" * 80)
print("PDF MATCH STATISTICS")
print("-" * 80)
match_counts = df_filtered['Match'].value_counts()
print(f"Records with matching PDFs (Yes): {match_counts.get('Yes', 0)}")
print(f"Records without matching PDFs (No): {match_counts.get('No', 0)}")
print(f"Match rate: {(match_counts.get('Yes', 0) / len(df_filtered) * 100):.2f}%")

print("\n" + "-" * 80)
print("NARRATIVE EXTRACTION STATISTICS")
print("-" * 80)
narratives_filled = (df_filtered['Narrative'] != '').sum()
narratives_empty = (df_filtered['Narrative'] == '').sum()
print(f"Rows with Narrative filled: {narratives_filled}")
print(f"Rows with Narrative empty: {narratives_empty}")
print(f"Percentage filled: {(narratives_filled / len(df_filtered) * 100):.2f}%")

print("\n" + "-" * 80)
print("BIKE TYPE DISTRIBUTION")
print("-" * 80)
bike_type_counts = df_filtered['Bike Type'].value_counts()
print(bike_type_counts)

print("\n" + "-" * 80)
print("MISSING VALUES SUMMARY")
print("-" * 80)
print(df_filtered.isnull().sum())

FINAL STATISTICS

Total records: 111
Unique HSMV Report Numbers: 111
Unique Bike Types: 10

--------------------------------------------------------------------------------
PDF MATCH STATISTICS
--------------------------------------------------------------------------------
Records with matching PDFs (Yes): 15
Records without matching PDFs (No): 96
Match rate: 13.51%

--------------------------------------------------------------------------------
NARRATIVE EXTRACTION STATISTICS
--------------------------------------------------------------------------------
Rows with Narrative filled: 14
Rows with Narrative empty: 97
Percentage filled: 12.61%

--------------------------------------------------------------------------------
BIKE TYPE DISTRIBUTION
--------------------------------------------------------------------------------
Bike Type
E-bike                  42
Motorized bicycle       37
Pedicab                 16
E-trike                  4
Bicycle with trailer     3
Police bike      

In [53]:
# Save to Excel
df_filtered.to_excel(output_file, index=False)
print(f"\n{'=' * 80}")
print(f"Data saved to: {output_file}")
print(f"{'=' * 80}")
print(f"\nColumns in output file:")
for col in df_filtered.columns:
    print(f"  - {col}")
print(f"\nTotal rows: {len(df_filtered)}")
print(f"\nNote: ZIP_Folder_Location column added to track source folders for matched PDFs")


Data saved to: bike_data_with_narratives.xlsx

Columns in output file:
  - HSMV_Report_Number
  - Bike Type
  - Narrative
  - Match
  - ZIP_Folder_Location

Total rows: 111

Note: ZIP_Folder_Location column added to track source folders for matched PDFs


In [54]:
# Display final dataframe
print("\nFinal DataFrame Preview:")
df_filtered.head(10)


Final DataFrame Preview:


,HSMV_Report_Number,Bike Type,Narrative,Match,ZIP_Folder_Location
403,82089001,Motorized bicycle,,No,
530,81994356,Motorized bicycle,,No,
745,82877472,E-trike,,No,
927,83350430,Pedicab,,No,
1031,83353372,Motorized bicycle,,No,
1205,83353605,Police bike,,No,
1487,83360086,Bicycle with trailer,,No,
1556,80823780,E-trike,,No,
2079,83968397,Adult tricycle,,No,
2100,83671693,Recumbent trike,,No,


In [55]:
# Examine the structure of both files
import pandas as pd

# Load bike_data_with_narratives.xlsx
bike_df = pd.read_excel('bike_data_with_narratives.xlsx')
print("bike_data_with_narratives.xlsx:")
print(f"Shape: {bike_df.shape}")
print(f"Columns: {list(bike_df.columns)}")
print(f"\nFirst few rows:")
print(bike_df.head())
print("\n" + "="*80 + "\n")

# Load label.xlsx
label_df = pd.read_excel('label.xlsx')
print("label.xlsx:")
print(f"Shape: {label_df.shape}")
print(f"Columns: {list(label_df.columns)}")
print(f"\nFirst few rows:")
print(label_df.head())

bike_data_with_narratives.xlsx:
Shape: (111, 5)
Columns: ['HSMV_Report_Number', 'Bike Type', 'Narrative', 'Match', 'ZIP_Folder_Location']

First few rows:
   HSMV_Report_Number          Bike Type Narrative Match ZIP_Folder_Location
0            82089001  Motorized bicycle       NaN    No                 NaN
1            81994356  Motorized bicycle       NaN    No                 NaN
2            82877472            E-trike       NaN    No                 NaN
3            83350430            Pedicab       NaN    No                 NaN
4            83353372  Motorized bicycle       NaN    No                 NaN


label.xlsx:
Shape: (2787, 12)
Columns: ['report_id', 'narrative', 'is_ebike_escooter', 'unsure', 'non-motorist type', 'notes', 'non_motorist_type', 'vehicle_body_type', 'non_motorist_location', 'non_motorist_action', 'posted_speed', 'estimated_speed']

First few rows:
    report_id                                          narrative  \
0  24012633.0  VEH 1 WAS AT N 40TH AV AND SH

In [56]:
# Check if original file has location/zip information
original_df = pd.read_excel('nolabeluse.xlsx')
print("nolabeluse.xlsx columns:")
print(list(original_df.columns))
print(f"\nLooking for location columns...")
location_cols = [col for col in original_df.columns if any(word in col.lower() for word in ['zip', 'location', 'city', 'county', 'address'])]
print(f"Potential location columns: {location_cols}")

# Show a sample if we found location columns
if location_cols:
    print(f"\nSample data from location columns:")
    print(original_df[['HSMV_Report_Number', 'Bike Type'] + location_cols].head(10))

nolabeluse.xlsx columns:
['HSMV_Report_Number', 'Reporting_Agency', 'Form_Type', 'Year', 'Crash_Date', 'Crash_Time', 'City', 'County', 'Crash_Street', 'Intersecting_Street', 'Vehicles', 'Non_Motorists', 'Fatalities', 'Injuries', 'Alcohol_Related', 'Distraction_Related', 'Drug_Related', 'Weather_Condition', 'Light_Condition', 'Crash_Severity', 'Type_of_Intersection', 'Road_Sys_Identifier', 'Type_of_Shoulder', 'Road_Surf_Cond', 'Bicyclists', 'Possible_Injuries', 'Non_Incapacitating_Injuries', 'Incapacitating_Injuries', 'Fatalities_30_Days', 'Non_Traffic_Fatalities', 'S4_Mapping', 'S4_Decimal_Degree_Longitude', 'S4_Decimal_Degree_Latitude', 'S4_Albers_X', 'S4_Albers_Y', 'S4_Mapping_Date', 'Bike_Crash_Group_Number', 'Bike_Crash_Group', 'Bike_Crash_Type_Number', 'Bike_Crash_Type', 'Bike_Crash_Location', 'Bike_Bicyclist_Direction', 'Bike_Bicyclist_Position', 'Bike_Typing_Notes', 'Bike Type']

Looking for location columns...
Potential location columns: ['City', 'County', 'Bike_Crash_Location'

## Task 1: Add Location (City & County) to bike_data_with_narratives.xlsx

In [57]:
# Prepare base bike data (no location columns)
bike_df_updated = pd.read_excel('bike_data_with_narratives.xlsx')

# Drop location columns if present
location_cols = [col for col in ['City', 'County'] if col in bike_df_updated.columns]
if location_cols:
    bike_df_updated = bike_df_updated.drop(columns=location_cols)

# Ensure ZIP_Folder_Location exists
if 'ZIP_Folder_Location' not in bike_df_updated.columns:
    bike_df_updated['ZIP_Folder_Location'] = ''

# Keep only the usual columns for final output
base_cols = ['HSMV_Report_Number', 'Bike Type', 'Narrative', 'Match', 'ZIP_Folder_Location']
bike_df_updated = bike_df_updated[base_cols]

# If not a match, clear ZIP_Folder_Location
bike_df_updated.loc[bike_df_updated['Match'] != 'Yes', 'ZIP_Folder_Location'] = ''

print("Base bike data (no location columns):")
print(f"Shape: {bike_df_updated.shape}")
print(f"Columns: {list(bike_df_updated.columns)}")
print(f"\nFirst 10 rows:")
print(bike_df_updated.head(10))

Base bike data (no location columns):
Shape: (111, 5)
Columns: ['HSMV_Report_Number', 'Bike Type', 'Narrative', 'Match', 'ZIP_Folder_Location']

First 10 rows:
   HSMV_Report_Number             Bike Type Narrative Match  \
0            82089001     Motorized bicycle       NaN    No   
1            81994356     Motorized bicycle       NaN    No   
2            82877472               E-trike       NaN    No   
3            83350430               Pedicab       NaN    No   
4            83353372     Motorized bicycle       NaN    No   
5            83353605           Police bike       NaN    No   
6            83360086  Bicycle with trailer       NaN    No   
7            80823780               E-trike       NaN    No   
8            83968397        Adult tricycle       NaN    No   
9            83671693       Recumbent trike       NaN    No   

  ZIP_Folder_Location  
0                      
1                      
2                      
3                      
4                      
5 

## Task 2: Combine label.xlsx with bike_data_with_narratives.xlsx

In [58]:
# Prepare label.xlsx data to match the structure (no location columns)
label_df_formatted = label_df[['report_id', 'non-motorist type', 'narrative']].copy()

# Rename columns to match bike_data structure
label_df_formatted.rename(columns={
    'report_id': 'HSMV_Report_Number',
    'non-motorist type': 'Bike Type',
    'narrative': 'Narrative'
}, inplace=True)

# Add Match and ZIP_Folder_Location columns
label_df_formatted['Match'] = 'n/a'
label_df_formatted['ZIP_Folder_Location'] = ''

# Drop rows with NaN in HSMV_Report_Number before converting to int
label_df_formatted = label_df_formatted.dropna(subset=['HSMV_Report_Number'])

# Convert report_id to integer to match HSMV_Report_Number format
label_df_formatted['HSMV_Report_Number'] = label_df_formatted['HSMV_Report_Number'].astype(int)

# Reorder columns to match final structure
label_df_formatted = label_df_formatted[['HSMV_Report_Number', 'Bike Type', 'Narrative', 'Match', 'ZIP_Folder_Location']]

print("Formatted label data:")
print(f"Shape: {label_df_formatted.shape}")
print(f"Columns: {list(label_df_formatted.columns)}")
print(f"\nFirst 10 rows:")
print(label_df_formatted.head(10))

Formatted label data:
Shape: (2680, 5)
Columns: ['HSMV_Report_Number', 'Bike Type', 'Narrative', 'Match', 'ZIP_Folder_Location']

First 10 rows:
   HSMV_Report_Number       Bike Type  \
0            24012633            bike   
1            24018175         scooter   
2            24018185         scooter   
3            24021620  motorized bike   
4            24027038          e-bike   
5            24027069       e-scooter   
6            24027114          e-bike   
7            24027139          e-bike   
8            24032662          e-bike   
9            24033091            bike   

                                           Narrative Match ZIP_Folder_Location  
0  VEH 1 WAS AT N 40TH AV AND SHERIDAN ST. VEH 1 ...   n/a                      
1  THE VICTIM WAS RIDING HIS SCOOTER EASTBOUND IN...   n/a                      
2  V1 WAS TRAVELING SOUTHBOUND ON N FEDERAL HWY I...   n/a                      
3  On Tuesday July 21, 2020 at 1424 Hrs. I respon...   n/a                     

In [59]:
# Combine bike_data_with_narratives with formatted label data
combined_df = pd.concat([bike_df_updated, label_df_formatted], ignore_index=True)

print("Combined data:")
print(f"Shape: {combined_df.shape}")
print(f"bike_data_with_narratives rows: {len(bike_df_updated)}")
print(f"label.xlsx rows: {len(label_df_formatted)}")
print(f"Total combined rows: {len(combined_df)}")
print(f"\nColumns: {list(combined_df.columns)}")
print(f"\nSample from bike_data_with_narratives (first 5 rows):")
print(combined_df.head())
print(f"\nSample from label.xlsx (rows {len(bike_df_updated)} to {len(bike_df_updated)+5}):")
print(combined_df.iloc[len(bike_df_updated):len(bike_df_updated)+5])

Combined data:
Shape: (2791, 5)
bike_data_with_narratives rows: 111
label.xlsx rows: 2680
Total combined rows: 2791

Columns: ['HSMV_Report_Number', 'Bike Type', 'Narrative', 'Match', 'ZIP_Folder_Location']

Sample from bike_data_with_narratives (first 5 rows):
   HSMV_Report_Number          Bike Type Narrative Match ZIP_Folder_Location
0            82089001  Motorized bicycle       NaN    No                    
1            81994356  Motorized bicycle       NaN    No                    
2            82877472            E-trike       NaN    No                    
3            83350430            Pedicab       NaN    No                    
4            83353372  Motorized bicycle       NaN    No                    

Sample from label.xlsx (rows 111 to 116):
     HSMV_Report_Number       Bike Type  \
111            24012633            bike   
112            24018175         scooter   
113            24018185         scooter   
114            24021620  motorized bike   
115            240

## Task 3: Classify Bike Types into 3 Groups

In [60]:
# First, let's see all unique bike types in the combined data
unique_bike_types = combined_df['Bike Type'].value_counts()
print("All unique bike types and their counts:")
print(unique_bike_types)
print(f"\nTotal unique bike types: {len(unique_bike_types)}")

All unique bike types and their counts:
Bike Type
e-scooter                              50
E-bike                                 42
Motorized bicycle                      37
e-bike                                 18
Pedicab                                16
bike                                   15
scooter                                14
motorized bike                          9
motorized scooter                       6
kick scooter                            5
E-trike                                 4
none                                    4
pedestrian                              3
gas bike                                3
skateboard                              3
Bicycle with trailer                    3
Tandem                                  2
moped                                   2
cyclist                                 2
unknown                                 2
Bike share                              2
Police bike                             2
pedecycle                 

In [61]:
# Create classification function for 3 groups
def classify_bike_type(bike_type):
    """
    Classify bike types into 3 categories:
    1. E-bike
    2. E-scooter
    3. Other (Non-motorists or Cyclists)
    """
    if pd.isna(bike_type):
        return 'Other (Non-motorists or Cyclists)'
    
    bike_type_lower = str(bike_type).lower()
    
    # E-bike: vehicles powered partly or entirely by electricity (motorized bikes and pedal-assisted bikes)
    ebike_keywords = [
        'e-bike', 'motorized bicycle', 'e-trike', 'electric bicycle', 
        'powered bike', 'toy e-bike', 'ebike'
    ]
    if any(keyword in bike_type_lower for keyword in ebike_keywords):
        return 'E-bike'
    
    # E-scooter: electric scooters, standing or seated, powered by battery
    escooter_keywords = [
        'e-scooter', 'motorized scooter', 'motorized standing scooter',
        'kick scooter', 'seated electric scooter', 'e or gas scooter',
        'e-skateboard', 'one-wheel'
    ]
    if any(keyword in bike_type_lower for keyword in escooter_keywords):
        return 'E-scooter'
    
    # Other: human-powered vehicles, cyclists, non-motorized bikes
    # This includes: bike, pedicab, bicycle with trailer, pedestrian, skateboard, police bike,
    # adult tricycle, bike share, tandem, eunicycle, pedecycle, cyclist, moped, recumbent trike,
    # tadpole tricycle, three wheel pedal scooter, tricycle, minibike, dirt bike, 
    # motorized wheelchair, ATV, etc.
    return 'Other (Non-motorists or Cyclists)'

# Apply classification
combined_df['Bike Type Group'] = combined_df['Bike Type'].apply(classify_bike_type)

# Show statistics for each group
print("Bike Type Classification:")
print("="*80)
group_counts = combined_df['Bike Type Group'].value_counts()
print(f"\n{group_counts}\n")
print("="*80)

# Show sample bike types in each group
for group in ['E-bike', 'E-scooter', 'Other (Non-motorists or Cyclists)']:
    print(f"\n{group}:")
    print("-" * 80)
    group_data = combined_df[combined_df['Bike Type Group'] == group]
    bike_types_in_group = group_data['Bike Type'].value_counts()
    print(f"Total records: {len(group_data)}")
    print(f"Unique bike types: {len(bike_types_in_group)}")
    print(f"\nTop bike types in this group:")
    print(bike_types_in_group.head(15))
    print()

Bike Type Classification:

Bike Type Group
Other (Non-motorists or Cyclists)    2622
E-bike                                103
E-scooter                              66
Name: count, dtype: int64


E-bike:
--------------------------------------------------------------------------------
Total records: 103
Unique bike types: 6

Top bike types in this group:
Bike Type
E-bike               42
Motorized bicycle    37
e-bike               18
E-trike               4
toy e-bike            1
powered bike          1
Name: count, dtype: int64


E-scooter:
--------------------------------------------------------------------------------
Total records: 66
Unique bike types: 8

Top bike types in this group:
Bike Type
e-scooter                     50
motorized scooter              6
kick scooter                   5
e or gas scooter               1
e-skateboard                   1
motorized standing scooter     1
one-wheel eskateboard          1
seated electric scooter        1
Name: count, dtype: int64

In [62]:
# Display a comprehensive summary of the classification
print("=" * 80)
print("FINAL BIKE TYPE CLASSIFICATION SUMMARY")
print("=" * 80)
print(f"\nTotal records: {len(combined_df)}")
print(f"\nBreakdown by group:")
print(f"  1. E-bike:                                 {len(combined_df[combined_df['Bike Type Group'] == 'E-bike']):>5} records ({len(combined_df[combined_df['Bike Type Group'] == 'E-bike'])/len(combined_df)*100:>5.1f}%)")
print(f"  2. E-scooter:                              {len(combined_df[combined_df['Bike Type Group'] == 'E-scooter']):>5} records ({len(combined_df[combined_df['Bike Type Group'] == 'E-scooter'])/len(combined_df)*100:>5.1f}%)")
print(f"  3. Other (Non-motorists or Cyclists):     {len(combined_df[combined_df['Bike Type Group'] == 'Other (Non-motorists or Cyclists)']):>5} records ({len(combined_df[combined_df['Bike Type Group'] == 'Other (Non-motorists or Cyclists)'])/len(combined_df)*100:>5.1f}%)")

print("\n" + "=" * 80)
print("Sample records from each group:")
print("=" * 80)

for group in ['E-bike', 'E-scooter', 'Other (Non-motorists or Cyclists)']:
    print(f"\n{group}:")
    sample = combined_df[combined_df['Bike Type Group'] == group][['HSMV_Report_Number', 'Bike Type', 'Match', 'ZIP_Folder_Location']].head(5)
    print(sample.to_string(index=False))
    
print("\n" + "=" * 80)
print("NOTE: The 'Bike Type Group' column has been added to the dataframe")
print("      but NOT saved to the Excel file yet (as requested).")
print("=" * 80)

FINAL BIKE TYPE CLASSIFICATION SUMMARY

Total records: 2791

Breakdown by group:
  1. E-bike:                                   103 records (  3.7%)
  2. E-scooter:                                 66 records (  2.4%)
  3. Other (Non-motorists or Cyclists):      2622 records ( 93.9%)

Sample records from each group:

E-bike:
 HSMV_Report_Number         Bike Type Match ZIP_Folder_Location
           82089001 Motorized bicycle    No                    
           81994356 Motorized bicycle    No                    
           82877472           E-trike    No                    
           83353372 Motorized bicycle    No                    
           80823780           E-trike    No                    

E-scooter:
 HSMV_Report_Number         Bike Type Match ZIP_Folder_Location
           24027069         e-scooter   n/a                    
           24065491 motorized scooter   n/a                    
           24066383         e-scooter   n/a                    
           24072808   

## Task 4: AI-Based Classification using Narratives

Use LLM to classify bike types from narrative text into one of 3 categories:
- E-bike
- E-scooter
- Other (Non-motorists or Cyclists)

In [ ]:
# Set up OpenAI client for UFL's LiteLLM Proxy
import openai

client = openai.OpenAI(
    api_key="sk-esFPwPZnoQ5rtPd5aNSE-A",
    base_url="https://api.ai.it.ufl.edu"  # LiteLLM Proxy is OpenAI compatible
)

print("OpenAI client configured for UFL's LiteLLM Proxy")
print("Model: gpt-4o-mini")

OpenAI client configured for UFL's LiteLLM Proxy
Model: llama-3.3-70b-instruct


In [ ]:
# Function to classify bike type from narrative using AI
def classify_with_ai(narrative):
    """
    Use LLM to classify the vehicle type from crash narrative
    Returns one of: 'E-bike', 'E-scooter', or 'Other (Non-motorists or Cyclists)'
    """
    if not narrative or pd.isna(narrative) or str(narrative).strip() == '':
        return 'Unknown - No narrative'
    
    # Create classification prompt
    prompt = f"""Read the following crash report narrative and classify the non-motorist vehicle involved into ONE of these three categories:

1. E-bike: Vehicles powered partly or entirely by electricity, including:
   - E-bikes (electric bicycles, pedal-assisted bikes)
   - Motorized bicycles
   - E-trikes (electric tricycles)
   - Electric bicycles
   - Powered bikes
   - Toy e-bikes

2. E-scooter: Electric scooters (standing or seated), typically powered by battery, including:
   - E-scooters
   - Motorized scooters
   - Motorized standing scooters
   - Kick scooters
   - Seated electric scooters
   - E-skateboards
   - One-wheel skateboards

3. Other (Non-motorists or Cyclists): Human-powered vehicles, cyclists, and non-motorized transportation, including:
   - Regular bikes/bicycles
   - Pedicabs
   - Bicycles with trailers
   - Pedestrians
   - Skateboards (non-electric)
   - Police bikes
   - Adult tricycles
   - Bike share bicycles
   - Tandems
   - Unicycles
   - Recumbent trikes
   - Traditional scooters (non-motorized)
   - Mopeds
   - Dirt bikes
   - ATVs
   - Wheelchairs

Crash Narrative:
{narrative}

Based on the narrative above, what type of vehicle was involved? Respond with ONLY one of these three options:
- E-bike
- E-scooter
- Other (Non-motorists or Cyclists)"""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            temperature=0.1,  # Low temperature for consistent classification
            max_tokens=50
        )
        
        # Extract the classification from response
        classification = response.choices[0].message.content.strip()
        
        # Ensure valid response
        valid_categories = ['E-bike', 'E-scooter', 'Other (Non-motorists or Cyclists)']
        if classification in valid_categories:
            return classification
        else:
            # Try to find the category in the response
            for category in valid_categories:
                if category.lower() in classification.lower():
                    return category
            # If not found, default to 'Other (Non-motorists or Cyclists)'
            return 'Other (Non-motorists or Cyclists)'
    except Exception as e:
        return f'Error - {str(e)}'

print("AI classification function defined!")

AI classification function defined!


In [ ]:
# Apply AI classification to all records with narratives
print("Applying AI classification to all records with narratives...")
print("=" * 80)

# Initialize the column if missing
if 'Final Bike Type' not in combined_df.columns:
    combined_df['Final Bike Type'] = ''

# Only classify rows with narratives
records_with_narratives = combined_df[combined_df['Narrative'].notna() & (combined_df['Narrative'] != '')]
target_indices = records_with_narratives.index
total = len(target_indices)
print(f"Total narratives to classify: {total}")

# Test the first narrative and stop if error
first_idx = target_indices[0] if total > 0 else None
if first_idx is not None:
    narrative = combined_df.loc[first_idx, 'Narrative']
    ai_classification = classify_with_ai(narrative)
    combined_df.loc[first_idx, 'Final Bike Type'] = ai_classification
    if isinstance(ai_classification, str) and ai_classification.startswith('Error -'):
        print(f"Error detected in first narrative: {ai_classification}")
        print("Stopping further classification.")
    else:
        # Proceed with the rest only if first is not error
        for i, idx in enumerate(target_indices[1:], 2):
            narrative = combined_df.loc[idx, 'Narrative']
            ai_classification = classify_with_ai(narrative)
            combined_df.loc[idx, 'Final Bike Type'] = ai_classification
            if i % 50 == 0 or i == total:
                print(f"Progress: {i}/{total}")

print("\n" + "=" * 80)
print("AI Classification Complete!")
print("=" * 80)
print(f"\nSummary:")
print(f"Total records: {len(combined_df)}")
print(f"Records with 'Final Bike Type' filled: {(combined_df['Final Bike Type'] != '').sum()}")
print(f"\nFirst 10 AI classifications:")
print(combined_df.loc[target_indices[:10], ['HSMV_Report_Number', 'Bike Type', 'Bike Type Group', 'Final Bike Type']])

# Save the combined file (single output) with AI results
combined_df.to_excel('NEW.xlsx', index=False)
print("\n✓ Saved combined data to NEW.xlsx")

Applying AI classification to all records with narratives...
Total narratives to classify: 2694
Progress: 50/2694
Progress: 100/2694
Progress: 150/2694
Progress: 200/2694
Progress: 250/2694
Progress: 300/2694
Progress: 350/2694
Progress: 400/2694
Progress: 450/2694
Progress: 500/2694
Progress: 550/2694
Progress: 600/2694
Progress: 650/2694
Progress: 700/2694
Progress: 750/2694
Progress: 800/2694
Progress: 850/2694
Progress: 900/2694
Progress: 950/2694
Progress: 1000/2694
Progress: 1050/2694
Progress: 1100/2694
Progress: 1150/2694
Progress: 1200/2694
Progress: 1250/2694
Progress: 1300/2694
Progress: 1350/2694
Progress: 1400/2694
Progress: 1450/2694
Progress: 1500/2694
Progress: 1550/2694
Progress: 1600/2694
Progress: 1650/2694
Progress: 1700/2694
Progress: 1750/2694
Progress: 1800/2694
Progress: 1850/2694
Progress: 1900/2694
Progress: 1950/2694
Progress: 2000/2694
Progress: 2050/2694
Progress: 2100/2694
Progress: 2150/2694
Progress: 2200/2694
Progress: 2250/2694
Progress: 2300/2694
Prog

In [66]:
# Display final dataframe structure with all new columns
print("=" * 80)
print("COMPLETE DATAFRAME STRUCTURE")
print("=" * 80)
print(f"\nColumns in combined_df:")
for i, col in enumerate(combined_df.columns, 1):
    print(f"  {i}. {col}")

print(f"\nTotal rows: {len(combined_df)}")
print(f"\nSample data (first 5 rows with all columns):")
print(combined_df.head())

print("\n" + "=" * 80)
print("Column Statistics:")
print("=" * 80)
for col in combined_df.columns:
    non_empty = (combined_df[col].notna() & (combined_df[col] != '')).sum()
    print(f"{col}: {non_empty}/{len(combined_df)} filled ({non_empty/len(combined_df)*100:.1f}%)")

COMPLETE DATAFRAME STRUCTURE

Columns in combined_df:
  1. HSMV_Report_Number
  2. Bike Type
  3. Narrative
  4. Match
  5. ZIP_Folder_Location
  6. Bike Type Group
  7. Final Bike Type

Total rows: 2791

Sample data (first 5 rows with all columns):
   HSMV_Report_Number          Bike Type Narrative Match ZIP_Folder_Location  \
0            82089001  Motorized bicycle       NaN    No                       
1            81994356  Motorized bicycle       NaN    No                       
2            82877472            E-trike       NaN    No                       
3            83350430            Pedicab       NaN    No                       
4            83353372  Motorized bicycle       NaN    No                       

                     Bike Type Group Final Bike Type  
0                             E-bike                  
1                             E-bike                  
2                             E-bike                  
3  Other (Non-motorists or Cyclists)             

In [69]:
# --- FINAL CLEANING AND SUMMARY ---
# 1. Create FINAL column:
def get_final_value(row):
    # If both Bike Type and Narrative are present, use Bike Type Group
    if pd.notna(row['Bike Type']) and str(row['Bike Type']).strip() != '' and pd.notna(row['Narrative']) and str(row['Narrative']).strip() != '':
        return row['Bike Type Group']
    # Otherwise, use AI column if available
    elif pd.notna(row['Final Bike Type']) and str(row['Final Bike Type']).strip() != '':
        return row['Final Bike Type']
    else:
        return ''

combined_df['FINAL'] = combined_df.apply(get_final_value, axis=1)

# 2. Remove rows with empty narratives
combined_df_cleaned = combined_df[combined_df['Narrative'].notna() & (combined_df['Narrative'].str.strip() != '')].copy()

# 3. Show comprehensive stats for cleaned data
print("\n" + "=" * 80)
print("FINAL CLEANED DATA STATISTICS")
print("=" * 80)
print(f"\nTotal rows after cleaning: {len(combined_df_cleaned)}")
print(f"Rows removed (empty narratives): {len(combined_df) - len(combined_df_cleaned)}")

print("\n" + "-" * 80)
print("FINAL COLUMN DISTRIBUTION")
print("-" * 80)
final_counts = combined_df_cleaned['FINAL'].value_counts()
print(f"\nTotal unique classes in FINAL: {combined_df_cleaned['FINAL'].nunique()}")
print("\nBreakdown by class:")
for class_name in final_counts.index:
    count = final_counts[class_name]
    percentage = (count / len(combined_df_cleaned)) * 100
    print(f"  {class_name:40s} {count:>6,} records ({percentage:>6.2f}%)")

print(f"\nTotal with classification: {final_counts.sum()}")
print(f"Total without classification (empty): {len(combined_df_cleaned) - final_counts.sum()}")

# 4. Save cleaned data to NEW.xlsx
combined_df_cleaned.to_excel('NEW.xlsx', index=False)
print("\n" + "=" * 80)
print("✓ Saved cleaned data to NEW.xlsx")
print("=" * 80)


FINAL CLEANED DATA STATISTICS

Total rows after cleaning: 2694
Rows removed (empty narratives): 97

--------------------------------------------------------------------------------
FINAL COLUMN DISTRIBUTION
--------------------------------------------------------------------------------

Total unique classes in FINAL: 11

Breakdown by class:
  Other (Non-motorists or Cyclists)         1,199 records ( 44.51%)
  E-scooter                                   951 records ( 35.30%)
  E-bike                                      536 records ( 19.90%)
  Unclear - Since the narrative mentions that Person #2 was "pedaling a scooter", it implies that the scooter is human-powered, not electric. Additionally, the make "GOTRAX" is known for producing both electric and non-electric scooters,      1 records (  0.04%)
  Unclear - Since the narrative mentions a "scooter" but does not specify that it is electric or motorized, and given the context of the crash and the fact that the person involved is refe

In [ ]:
# --- UPDATE CLASSIFICATION: EXCLUDE MOTOR-POWERED FROM E-BIKE/E-SCOOTER ---
# New rule: if "motor" appears in Bike Type, treat as Other (not E-bike/E-scooter)

def classify_bike_type_updated(bike_type):
    if pd.isna(bike_type):
        return 'Other (Non-motorists or Cyclists)'

    bike_type_str = str(bike_type).strip().lower()

    # If it explicitly mentions motor, classify as Other
    if 'motor' in bike_type_str:
        return 'Other (Non-motorists or Cyclists)'

    # E-bike: ONLY electric (no motor wording)
    ebike_keywords = [
        'e-bike', 'e bike', 'ebike', 'electric bicycle', 'electric bike',
        'pedal assist', 'pedal-assisted', 'electric trike', 'e-trike'
    ]
    if any(keyword in bike_type_str for keyword in ebike_keywords):
        return 'E-bike'

    # E-scooter: ONLY electric (no motor wording)
    escooter_keywords = [
        'e-scooter', 'e scooter', 'electric scooter', 'e-skateboard',
        'one-wheel', 'one wheel'
    ]
    if any(keyword in bike_type_str for keyword in escooter_keywords):
        return 'E-scooter'

    return 'Other (Non-motorists or Cyclists)'

# Update Bike Type Group using the new rules
combined_df['Bike Type Group'] = combined_df['Bike Type'].apply(classify_bike_type_updated)

# Rebuild FINAL column with updated Bike Type Group
combined_df['FINAL'] = combined_df.apply(get_final_value, axis=1)

# Recreate cleaned dataset (remove empty narratives)
combined_df_cleaned = combined_df[combined_df['Narrative'].notna() & (combined_df['Narrative'].str.strip() != '')].copy()

# Updated stats
print("\n" + "=" * 80)
print("UPDATED FINAL CLEANED DATA STATISTICS")
print("=" * 80)
print(f"\nTotal rows after cleaning: {len(combined_df_cleaned)}")
print(f"Rows removed (empty narratives): {len(combined_df) - len(combined_df_cleaned)}")

print("\n" + "-" * 80)
print("FINAL COLUMN DISTRIBUTION (UPDATED RULES)")
print("-" * 80)
final_counts = combined_df_cleaned['FINAL'].value_counts()
print(f"\nTotal unique classes in FINAL: {combined_df_cleaned['FINAL'].nunique()}")
print("\nBreakdown by class:")
for class_name in final_counts.index:
    count = final_counts[class_name]
    percentage = (count / len(combined_df_cleaned)) * 100
    print(f"  {class_name:40s} {count:>6,} records ({percentage:>6.2f}%)")

print(f"\nTotal with classification: {final_counts.sum()}")
print(f"Total without classification (empty): {len(combined_df_cleaned) - final_counts.sum()}")

# Save updated cleaned data
combined_df_cleaned.to_excel('NEW.xlsx', index=False)
print("\n" + "=" * 80)
print("✓ Saved updated cleaned data to NEW.xlsx")
print("=" * 80)


UPDATED FINAL CLEANED DATA STATISTICS

Total rows after cleaning: 2694
Rows removed (empty narratives): 97

--------------------------------------------------------------------------------
FINAL COLUMN DISTRIBUTION (UPDATED RULES)
--------------------------------------------------------------------------------

Total unique classes in FINAL: 11

Breakdown by class:
  Other (Non-motorists or Cyclists)         1,217 records ( 45.17%)
  E-scooter                                   938 records ( 34.82%)
  E-bike                                      531 records ( 19.71%)
  Unclear - Since the narrative mentions that Person #2 was "pedaling a scooter", it implies that the scooter is human-powered, not electric. Additionally, the make "GOTRAX" is known for producing both electric and non-electric scooters,      1 records (  0.04%)
  Unclear - Since the narrative mentions a "scooter" but does not specify that it is electric or motorized, and given the context of the crash and the fact that the